In [97]:
import http.client
import pandas as pd
import time
import requests
import os
import re
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from bs4 import BeautifulSoup

In [141]:
# コードブロックを取り除く
code_block_pattern = re.compile(r'`{3}.+?`{3}', flags=re.DOTALL)
code_pattern = re.compile(r'`{1}.+?`{1}', flags=re.DOTALL)
http_pattern = re.compile(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)")
img_pattern = re.compile(r'[.+?]')
hoge_pattern = re.compile(r"[!#*-\n]")

In [91]:
load_dotenv()
TOKEN = os.environ["ACCESS_TOKEN"]
URL = os.environ["ENDPOINT"]
KEY_WORD = "Ruby"

In [92]:
header = {'Authorization': f'Bearer {TOKEN}' }
url = URL
params = {"query": KEY_WORD}

In [93]:
res = requests.get(
    url,
    headers=header,
    params=params,
)

In [151]:
tmp = res.json()[12]['body']

In [87]:
soup = BeautifulSoup('', 'html.parser')

In [131]:
print(BeautifulSoup(tmp, "lxml").text)

## はじめに
果たしてQiitaの界隈にHR/HMリスナーがどのくらいいるのか分かりませんが…  
ごめんなさい、『Qiitaのタイトルに **[イングヴェイ](https://ja.wikipedia.org/wiki/%E3%82%A4%E3%83%B3%E3%82%B0%E3%83%B4%E3%82%A7%E3%82%A4%E3%83%BB%E3%83%9E%E3%83%AB%E3%83%A0%E3%82%B9%E3%83%86%E3%82%A3%E3%83%BC%E3%83%B3)** って**書きたいがため**』にこの記事を書いています。
あと、できればBGMとして[劇場版超速弾きロボ_マルム～スティンガー_の唄](https://www.nicovideo.jp/watch/sm29438884)を聴きながら読んでもらえると幸いです。

自分のスマホのBookアプリのライブラリを久々に見直したところ、数年前(2019年)に自分でEpub化したイングヴェイの『[Yng-WAY-俺のやり方(RELENTLESS THE MEMOIR)](https://www.amazon.co.jp/dp/4401638522/)』がありました。(発売は2013年)  
このデータは元々スキャンしたjpgしかなかったのですが、これをEpub化するのに当たって色々思い出したので備忘録として残しておこうと思います。

尚、結果としてはjpgをzip化したファイルが56MBだったものがepubだと7.2MBになったのでかなりダイエットになったと思います。

```bash
 56M -rw-r--r-- 1 kurokky  blacktree  56M  8月 30  2013 yjm.zip
7.6M -rw-r--r-- 1 kurokky  blacktree 7.6M  8月 16  2019 ymj.mobi
7.2M -rw-r--r-- 1 kurokky  blacktree 7.2M  8月 23  2019 ymj.epub
```

## きっかけ
私は日本でKindleが未発売の2009年頃にアメリカのAmazonで[Kindle 2](https://www.itmedia.co.jp/ebook/articles/1103/25/news031.

In [173]:
def qiita_body_transform(body):
    tmp = BeautifulSoup(body, "lxml").text
    # tmp_2 = code_block_pattern.sub('', tmp) 
    tmp_3 = http_pattern.sub('', tmp)
    tmp_4 = img_pattern.sub('', tmp_3)
    # tmp_5 = code_pattern.sub('', tmp_4)
    tmp_6 = hoge_pattern.sub('', tmp_4)
    return tmp_6

In [174]:
qiita_body_transform(tmp)

'Railsでは、`on:`を使うことで、バリデーション実行タイミングを指定できたり、カスタムコンテキストとして実行するかしないかを調整できます。\n\n\n\n`on:`で定義したカスタムコンテキストを`valid`等の引数に渡すことでバリデーションのチェックができます。\nRailsガイドでは`valid(:xxx)`の使い方が載っていますが、一度にカスタムコンテキストを複数指定する方法を書いておきたいと思います。\n\n 結論\n\n結論としては、配列で指定してあげれば良いです。\n\n```ruby:controller\ndef create\n  bookvalid([:xxx, :yyy])\n  \nend\n```\n\n```ruby:bookrb\nwith_options on: :xxx do\n  validates :title, presence: true\n  略\nend\n\nwith_options on: :yyy do\n  validates :content, presence: true\n  略\nend\n\n```\n\nこうすることで、`:xxx`と`:yyy`どちらのバリデーションチェックもかけることができます。\n\n 疑問に思ったこと\n\n配列で指定できるのは良かったのですが、\n「なんで配列でも機能するし、シンボルを１つ渡しただけでも機能するんだろう？」と思いました。\nなので、ソースコードを除いてみることにしました。\n\n Rails内の仕組み\n\n初級者のため読むのにかなり苦労しました。ざっくりしかわかっていない点はご容赦ください、、、\n\n1まず、`valid`メソッドの中で`selfvalidation_context`に、引数をそのまま渡しています。（上記の例だと`[:xxx, :yyy]`）\n\n```ruby:gems/activemodel606/lib/active_model/validationsrb 333行目あたり\ndef valid(context = nil)\n  current_context, selfvalidation_context = validation_context, context\n  errorsclear\n  run_valid

In [155]:
content = res.json()

In [156]:
df = pd.json_normalize(content)

In [163]:
extract_df = df[["body","tags","title"]]

In [175]:
extract_df["transform_body"] =  extract_df["body"].map(qiita_body_transform)

/Users/naokitakahashi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [176]:
extract_df

,body,tags,title,transform_body
0,# 何がしたい？\n\neach文で回した親要素に含まれる子要素の順番をランダムにシャッフル...,"[{'name': 'Rails', 'versions': []}, {'name': '...",【Rails】each文内の要素をjQueryでシャッフル表示させる方法,何がしたい？\n\neach文で回した親要素に含まれる子要素の順番をランダムにシャッフルさ...
1,# はじめに\n\n文系出身で（ほぼ）未経験から、Webエンジニア職に就職した僕が入社後半年...,"[{'name': 'PHP', 'versions': []}, {'name': 'We...",Web系エンジニア新卒１年目が入社半年後に感じた、初学者エンジニアに必要なスキルまとめ,はじめに\n\n文系出身で（ほぼ）未経験から、Webエンジニア職に就職した僕が入社後半年経...
2,## はじめに\n果たしてQiitaの界隈にHR/HMリスナーがどのくらいいるのか分かりませ...,"[{'name': 'ImageMagick', 'versions': []}, {'na...",【俺のやり方】【OCR】イングヴェイの自伝をGoogleのCloud Vision APIを...,はじめに\n果たしてQiitaの界隈にHR/HMリスナーがどのくらいいるのか分かりませんが...
3,## はじめに\n[【Rails】Twitterクローンを作ってみた 〜コメント機能の作成〜...,"[{'name': 'HTML', 'versions': []}, {'name': 'R...",【Rails】Twitterクローンを作ってみた 〜画像アップロード(ActiveRecord)〜,はじめに\n[【Rails】Twitterクローンを作ってみた 〜コメント機能の作成〜](...
4,# はじめに\n\nApplication Stream(AppStream)を利用してAl...,"[{'name': 'Ruby', 'versions': []}, {'name': 'a...",AlmaLinux OS 9にRuby 3.1をインストール（AppStream）,はじめに\n\nApplication Stream(AppStream)を利用してAlm...
5,# はじめに\n\nApplication Stream(AppStream)を利用してAl...,"[{'name': 'Ruby', 'versions': []}, {'name': 'a...",AlmaLinux OS 9にNodejs 18をインストール（AppStream）,はじめに\n\nApplication Stream(AppStream)を利用してAlm...
6,# はじめに\n\nApplication Stream(AppStream)を利用してAl...,"[{'name': 'Ruby', 'versions': []}, {'name': 'a...",AlmaLinux OS 8にRuby 3.1をインストール（AppStream）,はじめに\n\nApplication Stream(AppStream)を利用してAlm...
7,# はじめに\n文字列や配列を格納している変数の中に、要素が存在しているか確認するケースがあ...,"[{'name': 'Ruby', 'versions': []}, {'name': 'R...",文字列や配列等のオブジェクト内が空かどうか確かめるためのメソッド,はじめに\n文字列や配列を格納している変数の中に、要素が存在しているか確認するケースがあり...
8,# はじめに\nrails初学者の者です。オンラインでプログラミングを学習させて頂いてるので...,"[{'name': 'Ruby', 'versions': []}, {'name': 'R...",【Rails】RspecのFormオブジェクトモデル単体テストコードのエラー,はじめに\nrails初学者の者です。オンラインでプログラミングを学習させて頂いてるのです...
9,# はじめに\nrails初学者の者です。オンラインでプログラミングを学習させて頂いてるので...,"[{'name': 'Ruby', 'versions': []}, {'name': 'R...","【RUBY】if,elseを用いたプログラム",はじめに\nrails初学者の者です。オンラインでプログラミングを学習させて頂いてるのです...
